### Facial Recognition Function

In [ ]:
import nuclio

In [ ]:
%run set_env.ipynb

In [ ]:
# nuclio: start-code

In [ ]:
import cv2
from cv2 import VideoWriter_fourcc, VideoWriter
import sys
import os
import requests
import base64
import numpy as np

def stream_frame_write(context,frame,shard):
    headers = {
            "Content-Type": "application/json",
            "X-v3io-function": "PutRecords",
            "X-v3io-session-key" : os.getenv("V3IO_ACCESS_KEY")
          }
    data = base64.b64encode(frame)
    payload = {
            "Records": [
                {
                    "Data":  data.decode('utf-8'),
                    "ShardId" : shard
                    }
                ]
          }
    r = requests.post(os.getenv("TAGGED_VIDEO_STREAM_URL"), headers=headers,json=payload)   
    return r.text

def handler(context, event):
    shardId = event.shard_id
    event_data=event.body
    
    # Load the cascade
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # To capture video from webcam.
    img = event_data
    img = np.frombuffer(img, dtype=np.uint8)
    img = cv2.imdecode(img, flags=1)
    if bool(os.getenv('ROTATE_180')):
        img = cv2.rotate(img, cv2.ROTATE_180) # For some reason my stream is flipped?
    context.logger.info(img.shape)
    
    faces = face_cascade.detectMultiScale(img, 1.1, 4)
    
    # Draw the rectangle around each face
    face_count=0
    for (x, y, w, h) in faces:
        face_count += 1
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
        
    context.logger.info("Face Count: %s"% face_count )
    if face_count > 0 :
        ret, buffer = cv2.imencode('.jpg', img)
        try:
            r = stream_frame_write(context,buffer.tobytes(),shardId)
        except:
            context.logger.error("Failed to save frame for shard %s"% shardId)

In [ ]:
# nuclio: end-code

In [ ]:
from mlrun import code_to_function, mount_v3io, mlconf
import os

fn = code_to_function('deploy-facial-recognition', kind='nuclio', handler="handler", project=os.getenv("PROJECT"))

In [ ]:
fn.with_http(workers=1).apply(mount_v3io())
fn.spec.base_spec['spec']['build']['baseImage'] = f"docker-registry.{os.getenv('IGZ_NAMESPACE_DOMAIN')}:80/{os.getenv('DOCKER_IMAGE')}"
fn.spec.base_spec['spec']['loggerSinks'] = [{'level': 'info'}]
fn.spec.min_replicas = 1
fn.spec.max_replicas = 1

fn.set_env('V3IO_ACCESS_KEY', os.getenv('V3IO_ACCESS_KEY'))
fn.set_env('IGZ_CONTAINER', IGZ_CONTAINER)
fn.set_env('ROTATE_180', ROTATE_180)

trigger_spec={
      'kind': 'v3ioStream',
      'url' : "http://v3io-webapi:8081/%s/%s@processorgrp"% (os.getenv('IGZ_CONTAINER'),os.getenv('RAW_VIDEO_STREAM')),
    "password": os.getenv('V3IO_ACCESS_KEY'),  
    'attributes': {"pollingIntervalMs": 500,
        "seekTo": "earliest",
        "readBatchSize": 100,
        "partitions": "0-100",               
                     
      }
    }
fn.add_trigger('image-proc',trigger_spec)

fn.export("components/yaml/deploy-facial-recognition.yaml")